In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =======

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [2]:
import os
import glob

import numpy as np
import pandas as pd

import cudf
import cupy as cp
import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

### Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = 10000000
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., NUM_ROWS).astype(np.int32), 1, 50000)

# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, NUM_ROWS), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['timestamp/age_days'] = np.random.uniform(0, 1, NUM_ROWS).astype(np.float32)
df['timestamp/weekday/sin']= np.random.uniform(0, 1, NUM_ROWS).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [5]:
df.head()

,session_id,item_id,category,timestamp/age_days,timestamp/weekday/sin,day
0,88303,4,1,0.627299,0.059239,2
1,79291,15,2,0.693606,0.128668,2
2,75485,37,5,0.062722,0.111661,7
3,85283,42,5,0.744100,0.480346,1
4,84407,5,1,0.622424,0.989467,3


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers.  Note that in the `Categorify` op, we set `start_index=1`; the reason for that is, we want the encoded null values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

Here our goal is to create sequential features.  In this cell, we are creating temporal features and grouping them together at the session level, sorting the interactions by time. Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [6]:
# Categorify categorical features
categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify(start_index=1)

# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'timestamp/age_days', 'timestamp/weekday/sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (groupby_features['category-list']) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(0,20) 
    >> nvt.ops.Rename(postfix = '_trim')
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['timestamp/age_days-list', 'timestamp/weekday/sin-list'] 
    >> nvt.ops.ListSlice(0,20) 
    >> nvt.ops.Rename(postfix = '_trim')
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)


workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(df, cpu=False)
# Generate statistics for the features
workflow.fit(dataset)
# Apply the preprocessing and return an NVTabular dataset
sessions_ds = workflow.transform(dataset)
# Convert the NVTabular dataset to a Dask cuDF dataframe (`to_ddf()`) and then to cuDF dataframe (`.compute()`)
sessions_gdf = sessions_ds.to_ddf().compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [7]:
sessions_gdf.head(3)

,item_id-count,day-first,session_id,item_id-list_trim,category-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,586,5,2,"[13, 8, 39, 14, 23, 2, 4, 83, 25, 34, 17, 4, 1...","[3, 3, 6, 2, 4, 2, 2, 11, 4, 5, 4, 2, 15, 18, ...","[0.3265768, 0.41545194, 0.52078074, 0.7723212,...","[0.5636411, 0.14788395, 0.6995017, 0.010999571..."
1,586,6,3,"[20, 14, 8, 15, 10, 56, 73, 22, 18, 52, 27, 42...","[4, 2, 3, 3, 3, 8, 10, 4, 4, 7, 5, 6, 2, 9, 9,...","[0.13561419, 0.035071343, 0.58149755, 0.159483...","[0.3359673, 0.6002685, 0.84561634, 0.04078535,..."
2,584,2,4,"[24, 12, 178, 23, 23, 218, 92, 5, 55, 85, 10, ...","[4, 3, 22, 4, 4, 28, 12, 2, 8, 11, 3, 3, 4, 3,...","[0.39440218, 0.12561888, 0.27249986, 0.6201667...","[0.90376323, 0.75177085, 0.6668168, 0.0828298,..."


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [8]:
workflow.save('workflow_etl')

The following will generate `schema.pbtxt` file in the provided folder.

In [9]:
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [10]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))
!mkdir -p $OUTPUT_DIR

In [11]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|██████████| 9/9 [00:01<00:00,  5.79it/s]


## Checking the preprocessed outputs

In [12]:
TRAIN_PATHS = sorted(glob.glob(os.path.join(OUTPUT_DIR, "1", "train.parquet")))

In [13]:
gdf = cudf.read_parquet(TRAIN_PATHS[0])
gdf

,item_id-count,session_id,item_id-list_trim,category-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,578,8,"[17, 3, 130, 56, 35, 4, 12, 48, 21, 6, 13, 12,...","[4, 2, 17, 8, 5, 2, 3, 7, 2, 2, 3, 3, 3, 6, 2,...","[0.91336805, 0.7539333, 0.6858618, 0.91122335,...","[0.4654125, 0.8024907, 0.15122412, 0.9189323, ..."
1,570,24,"[102, 5, 28, 12, 7, 11, 34, 2, 21, 11, 20, 4, ...","[13, 2, 5, 3, 3, 3, 5, 2, 2, 3, 4, 2, 5, 13, 4...","[0.16131416, 0.7624795, 0.5117769, 0.06776055,...","[0.17060736, 0.23287642, 0.5058551, 0.28743693..."
2,567,32,"[11, 24, 2, 32, 6, 26, 14, 4, 5, 17, 17, 10, 8...","[3, 4, 2, 5, 2, 4, 2, 2, 2, 4, 4, 3, 3, 4, 2, ...","[0.62564784, 0.997358, 0.8010653, 0.027112987,...","[0.6030678, 0.25616208, 0.9580145, 0.99706334,..."
4,565,43,"[18, 16, 22, 32, 13, 73, 5, 21, 11, 61, 78, 73...","[4, 3, 4, 5, 3, 10, 2, 2, 3, 8, 10, 10, 6, 4, ...","[0.7378176, 0.16968544, 0.7315238, 0.95425814,...","[0.34397638, 0.8673334, 0.55496854, 0.9816106,..."
5,563,57,"[65, 15, 85, 11, 7, 60, 46, 11, 9, 48, 2, 19, ...","[9, 3, 11, 3, 3, 8, 7, 3, 2, 7, 2, 4, 7, 3, 2,...","[0.84140944, 0.032678757, 0.5808043, 0.9555968...","[0.1885734, 0.68167686, 0.5295532, 0.85896724,..."
...,...,...,...,...,...,...
2152,438,19951,"[8, 25, 12, 63, 33, 40, 22, 28, 77, 33, 13, 8,...","[3, 4, 3, 9, 2, 6, 4, 5, 10, 2, 3, 3, 2, 5, 2,...","[0.8278093, 0.15857665, 0.36844572, 0.19620946...","[0.1730732, 0.8465068, 0.15297464, 0.46283653,..."
2153,437,19961,"[23, 40, 77, 89, 33, 38, 19, 16, 13, 60, 11, 5...","[4, 6, 10, 12, 2, 6, 4, 3, 3, 8, 3, 2, 2, 8, 6...","[0.34355995, 0.16667433, 0.3479699, 0.6915854,...","[0.7232413, 0.54277384, 0.9209216, 0.15295857,..."
2155,436,19970,"[4, 37, 9, 87, 11, 18, 111, 64, 6, 7, 3, 22, 4...","[2, 6, 2, 12, 3, 4, 15, 9, 2, 3, 2, 4, 2, 4, 2...","[0.86520904, 0.2337783, 0.7927252, 0.60708684,...","[0.37351412, 0.9064367, 0.78618735, 0.14689812..."
2156,434,19977,"[4, 19, 58, 28, 11, 140, 3, 93, 32, 42, 7, 5, ...","[2, 4, 8, 5, 3, 18, 2, 12, 5, 6, 3, 2, 4, 3, 3...","[0.9443977, 0.50952923, 0.7083005, 0.6272203, ...","[0.01576173, 0.5609052, 0.9132833, 0.72388875,..."


You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model.